### Skeleton download

In [ ]:
# Install cloud-volume to access h01 mesh and skeleton data.
# You may need to restart the runtime after installing.
!pip -q install --upgrade numpy
!pip -q install cloud-volume

<span style="color: orange;">⚠️ Need to restart runtime after installing</span>


In [ ]:
import os
import numpy as np
import google.colab.auth
import cloudvolume
import asyncio
from google.colab import drive

In [ ]:
google.colab.auth.authenticate_user()
drive.flush_and_unmount
drive.mount("/content/drive", force_remount=True)

In [ ]:
c2_cloudvolume = cloudvolume.CloudVolume('gs://h01-release/data/20210601/c2', progress=True)

### Install SWC Directly


In [ ]:
IDs = [3166504661, 3240926264, 3284612002, 3327903858, 3400048815]
drive_root = '/content/drive/'
output_folder = f"{drive_root}/ImageGeneration/AllSWCs"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def write_skeleton(x, skel):
    swc = skel.to_swc()
    swc_path = f"{output_folder}/{x}.swc"

    with open(swc_path, 'w') as f:
        f.write(swc)

async def process_id(x, skel):
    write_skeleton(x, skel)

async def main():
    skels = c2_cloudvolume.skeleton.get(IDs)
    tasks = [process_id(x, skel) for x, skel in zip(IDs, skels)]
    await asyncio.gather(*tasks)

await main()

### Install as \[Vertex, Edge, Radius\]

In [ ]:
IDs = [3166504661, 3240926264, 3284612002, 3327903858, 3400048815]
drive_root = '/content/drive/'
output_folder = f"{drive_root}/ImageGeneration/AllSWCs"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

async def write_skeleton_files(skel, path_to):
    skv = path_to + '/skv{0}.txt'
    ske = path_to + '/ske{0}.txt'
    skr = path_to + '/skr{0}.txt'
    
    v = np.array(skel.vertices)
    e = np.array(skel.edges)
    r = np.array(skel.radius)
    
    np.savetxt(skv.format(skel.id), v.astype(float), fmt='%.2f')
    np.savetxt(ske.format(skel.id), e.astype(int), fmt='%i')
    np.savetxt(skr.format(skel.id), r.astype(float), fmt='%.2f')

async def process_id(x, skel):
    await write_skeleton_files(skel, output_folder)

async def main():
    skels = c2_cloudvolume.skeleton.get(IDs)
    tasks = [process_id(x, skel) for x, skel in zip(IDs, skels)]
    await asyncio.gather(*tasks)

await main()